## Overview

This is a project for automating trading. I am using Twelve Data API to access stock data. My algorithms will probably rely on technical analysis rather than fundamental analysis: data about the stocks movements rather than company releases, as I can automate this stuff easily.

## TwelveData API Tools

[Here is the documentation page for the API.](https://twelvedata.com/docs)

[Here is a tutorial for a library.](https://twelvedata.com/blog/get-high-quality-financial-data-directly-into-python)

[Here is the library's GitHub repository.](https://github.com/twelvedata/twelvedata-python)


I'm allowed up to 800 calls per day.


## Setup

The first steps in this project are setting up this venv with my desired libraries. I am installing: 

>`pip install twelvedata[matplotlib, plotly]`


## First Steps

I will begin by practicing fetching data from the API and displaying it. This will help me using tools to graphically represent data.

In [ ]:
from twelvedata import TDClient
import json
import datetime

# Construct the necessary time series
ts = td.time_series(
    symbol="AAPL",
    interval="1min",
    timezone="America/New_York",
    outputsize=1
).as_json()

# with open("results.txt", "w") as f: 
#     f.write(ts.to_string())

with open("results.txt", "w") as f: 
    f.write(json.dumps(ts, indent=4))


## First Algorithm

This algorithm will buy some AAPL stock if the price is below a certain number. 

First, I will create the sandbox that keeps track of my possesions. I will have the API connection to the external world as a given. Second, I will work on using the API to interact with the world to get the data I want.

As I go forward, I will iterate over the first step to improve the algorithm. 

Stipulations:
* 2023-04-04 8:11 AM: The API returns a range of prices of an asset over a time interval. I will take the current price to be the open.

In [50]:
"""
This is the basic structure for the trading algorithm. This creates the environment that manages possessions and transactions. That occurs client-side. 


The API is contained in the function that fetches data. Everything else occus on the machine.

"""

from twelvedata import TDClient

# This is accessible to all functions
global funds_usd 
funds_usd = 500

global assets
assets = []

def find_index(stock_ticker):
    """
    Find the index of a stock in the `assets` list. Return -1 if not found.

    Argument:
        stock_ticker (string)

    Return:
        index (int)
    """
    for i in assets:
        if assets[i]["symbol"] == stock_ticker:
            return i
    else:
        return -1

# Initialize the API
td = TDClient(apikey="154d1e9a3af04dc4a9db8b9172cdebbc")  


# This bridges this algorithm to the stock market.
def fetch_stock_data(stock_ticker):
    """
    Gets the necessary data from the API.

    Argument:
        stock_ticker (string): ticker 

    Return (dict):
        price per share at time of call
    """
    td = TDClient(apikey="154d1e9a3af04dc4a9db8b9172cdebbc")  
    # Construct the time series
    ts = td.time_series(
        symbol=stock_ticker,
        interval="1min",
        timezone="America/New_York",
        outputsize=1
    ).as_json()
    return ts

# Execute calculations
def decide_buy(stock_ticker):
    """
    Decide whether to buy a stock

    Argument: 
        stock_ticker (string)

    Return:
        choice (boolean)
    """
    return

def decide_sell(stock_ticker):
    """
    Decide whether to sell a stock

    Argument: 
        stock_ticker (string)

    Return:
        choice (boolean)
    """
    return


# Keep count of assets
def exec_purchase(stock_ticker, shares, price_per_share):
    """
    Execute a purchase through the API.

    Argument:
        stock_ticker (string)
        shares (float): The amount of shares to buy 

    Return:
        (boolean)
    """
    value = shares * price_per_share
    if value < funds_usd:
        raise Exception(f"Purchase (${value}) exceeds available funds (${funds_usd}).")
    # Spend money
    funds_usd -= value
    # Update possessions
    idx = find_index(stock_ticker)
    if idx == -1:
        assets.append({
            "symbol": stock_ticker,
            "shares": shares,
            "price": price_per_share
        })
    else:
        assets[idx]["shares"] += shares
        assets[idx]["price"] += price_per_share
    return True

def exec_sell(stock_ticker, shares, price_per_share):
    """
    Sell shares through the API.

    Argument:
        stock_ticker (string)
        shares (float): The amount of shares to sell  

    Return:
        (boolean): success or failure
    """
    value = shares * price_per_share
    idx = find_index(stock_ticker)
    if idx == -1:
        raise Exception(f"Attempted to sell nonexistent asset: {stock_ticker}.")
    else: 
        # Ensure transaction is valid
        if shares > assets[idx]["shares"]:
            raise Exception(f"Attempted to sell more shares than owned: {shares} > {assets[idx]['shares']}")
        # Excecute transaction
        assets[idx]["shares"] -= shares
        funds_usd += shares*price_per_share
        # Clean list
        if assets[idx]["shares"] == 0:
            del assets[idx]
    return True 




In [45]:
print(json.dumps(fetch_stock_data("AAPL"), indent=4))

[
    {
        "datetime": "2023-04-03 15:59:00",
        "open": "166.21001",
        "high": "166.25999",
        "low": "166.07001",
        "close": "166.10001",
        "volume": "893465"
    }
]
